In [ ]:
import os
from datacat4ml.Scripts.model_dev.ml_models import RF, GB, SVM, KNN
from datacat4ml.Scripts.model_dev.ml_tune import get_config, optuna_hpo
from datacat4ml.Scripts.model_dev.ml_dataloader import MLData
from datacat4ml.Scripts.model_dev.ml_bmk import algo_dict
from datacat4ml.const import ML_HP_DIR
from datacat4ml.const import FEAT_HHD_OR_DIR, FEAT_MHD_effect_OR_DIR, FEAT_MHD_OR_DIR, FEAT_LHD_OR_DIR

# Post-processing

In [ ]:
# read `bmk*csv' files.

# Test code

## optuna-hpo

In [ ]:
#----model----
model = RF # KNN, GB, RF, SVM
#----config----
config = get_config(os.path.join(ML_HP_DIR, f'{model.__name__}.json'))
print(f'config: \n{config}')
#----prepare data: inner_splits, x, y ----
#fpath = os.path.join(FEAT_HHD_OR_DIR, 'rmvD0', 'CHEMBL233_None_None_Ki_None_hhd_b50_b50_ECFP4.pkl')
fpath = os.path.join(FEAT_LHD_OR_DIR, 'rmvD0', 'CHEMBL233_agon_G-GTP_EC50_CHEMBL3888720_lhd_b50_b50_featurized.pkl')
# fpath = os.path.join(FEAT_DATA_DIR, 'old', 'feat_lhd_or', 'rmvDupMol0', 'CHEMBL233_agon_G-GTP_EC50_CHEMBL3888720_lhd_b50_b50_featurized.pkl') => can't use the old files due to the altered split columns.
data = MLData(fpath)
data(descriptor='ECFP4', rmvS=0, aim='vs', spl='rs_lo')
data.get_int_splits()

inner_splits = data.int_inner_splits
x = data.x
y = data.y

#----- run optuna_hpo -----
best_hparams, best_value = optuna_hpo(config, model, inner_splits, x, y, metric='auroc')
print(f'Best hyperparameters: {best_hparams}')
print(f'Best mean AUROC from inner CV: {best_value}')
print(f'inner_splits: {inner_splits},\nx: {x},\ny: {y}')

[I 2025-10-31 16:38:16,769] A new study created in memory with name: no-name-c1cde081-8b55-4bc6-aca7-cb7732649789


config: 
{'n_estimators': [100, 250, 500, 1000]}
x length: 64
y length: 64
activity_ids length: 64
col_prefix: int.rmvS0_rs_lo
col_names: ['int.rmvS0_rs_lo_fold0', 'int.rmvS0_rs_lo_fold1', 'int.rmvS0_rs_lo_fold2', 'int.rmvS0_rs_lo_fold3', 'int.rmvS0_rs_lo_fold4']

Number of outer splits found: 5
Generating outer splits for int.rmvS0_rs_lo...
Unique classes in outer_y_train: [0. 1.], counts: [25 26]
minClass_count: 25
Unique classes in outer_y_train: [0. 1.], counts: [25 26]
minClass_count: 25
Unique classes in outer_y_train: [0. 1.], counts: [26 25]
minClass_count: 25
Unique classes in outer_y_train: [0. 1.], counts: [26 25]
minClass_count: 25
Unique classes in outer_y_train: [0. 1.], counts: [26 26]
minClass_count: 26
max_minClass_count: 26
Generating inner splits for int.rmvS0_rs_lo...
min_class_count: 26
Inner train folds: [[0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 14, 15, 17, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51], [1, 

0it [00:00, ?it/s]


Fold 1/5 for RF
Creating the model...
Model created.
Training the model...


1it [00:00,  1.68it/s]

...

metric is auroc, Fold 1 AUROC: 0.4666666666666667

Fold 3/5 for RF
Creating the model...
Model created.
Training the model...
Model trained.
Validating the model...
Model validated.
metric is auroc, Fold 2 AUROC: 0.6799999999999999


4it [00:00,  5.17it/s]


Fold 4/5 for RF
Creating the model...
Model created.
Training the model...
Model trained.
Validating the model...
Model validated.
metric is auroc, Fold 3 AUROC: 0.92

Fold 5/5 for RF
Creating the model...
Model created.
Training the model...


5it [00:00,  5.16it/s]
[I 2025-10-31 16:38:27,646] Trial 3 finished with value: 0.6966666666666667 and parameters: {'n_estimators': 100}. Best is trial 3 with value: 0.6966666666666667.


Model trained.
Validating the model...
Model validated.
metric is auroc, Fold 4 AUROC: 0.8
Best hyperparameters: {'n_estimators': 100}
Best mean AUROC from inner CV: 0.6966666666666667
inner_splits: [{'inner_train_idx': [1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 14, 16, 18, 20, 26, 27, 28, 29, 30, 32, 33, 37, 38, 39, 40, 41, 42, 44, 45, 47, 48, 50, 51, 52, 53, 55, 56, 58, 60, 61, 63], 'inner_valid_idx': [10, 12, 15, 19, 23, 24, 25, 31, 34, 35, 43]}, {'inner_train_idx': [2, 5, 6, 7, 10, 11, 12, 13, 14, 15, 18, 19, 20, 23, 24, 25, 26, 28, 29, 30, 31, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 45, 47, 48, 50, 52, 55, 56, 58, 60, 61], 'inner_valid_idx': [1, 3, 4, 9, 16, 27, 32, 44, 51, 53, 63]}, {'inner_train_idx': [1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 13, 15, 16, 19, 20, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 51, 52, 53, 56, 58, 60, 63], 'inner_valid_idx': [5, 14, 18, 26, 29, 30, 48, 50, 55, 61]}, {'inner_train_idx': [1, 3, 4, 5, 7, 9, 10, 12, 13, 14, 15, 16, 18, 1

## Pipeline

In [ ]:
# for `filepath` 
ds_path = ['feat_hhd_or', 'feat_mhd-effect_or', 'feat_mhd_or', 'feat_lhd_or']
rmvDs = [0, 1] # when rmvD1, the test set in parent file would be possibly more than that in the child file.

# for `filename`
targets = ['CHEMBL233', 'CHEMBL237', 'CHEMBL236', 'CHEMBL2014']
effects = ['None', 'bind', 'agon', 'antag']
assays = ['None', 'RBA', 'G-GTP', 'G-cAMP', 'G-Ca', 'B-arrest']
standard_types = ['None', 'Ki', 'IC50', 'EC50']
assay_chembl_ids = ['None', 'CHEMBL867087', 'CHEMBL1030622', 'CHEMBL3888830'] # just name a few here

# for x
descriptors = ['ECFP4', 'ECFP6', 'MACCS', 'RDKITFP', 'PHARM2D', 'ERG',
               'PHYSICOCHEM',
               'SHAPE3D', 'AUTOCORR3D', 'RDF', 'MORSE', 'WHIM', 'GETAWAY'
               ]
# for y
aims = ['vs', 'lo']

# for train/test split
SPLs = ['int', 'aln']

# internal split (e.g. int.
#                      rmvS0_cs_fold0)
rmvSs = [0, 1] # when rmvS1, the test set in parent file would be possibly less than that in the child file.
spls = ['rs-lo', 'rs-vs', 'cs', 'ch']

# aligned split (e.g. aln. => Yu: need to carefully consider the pipeline here.
#                     child.
#                     hhd.mhd.
#                     CHEMBL233_None_None_Ki_None.
#                     CHEMBL233_bind_RBA_Ki_None.
#                     rmvS0_rs-lo_fold0)
positions = ['parent', 'child']
pairs = ['hhd.mhd', 'hhd.lhd', 'mhd-effect.mhd', 'mhd-effect.lhd', 'mhd.lhd']
parent_filenames = []
child_filenames = []

#YU if:
int_col = ['int.rmvS0_rs-lo', 'int.rmvS0_rs-vs', 'int.rmvS0_cs', 'int.rmvS0_ch',
           'int.rmvS1_rs-lo', 'int.rmvS1_rs-vs', 'int.rmvS1_cs', 'int.rmvS1_ch']
aln_col = ['aln.child.hhd.mhd.pfname.cfname.int_col']
aln_col = ['aln.child.pfname.cfname.int_col']

# for pipelines => Yu: consider a better name
pipelines = ['simpleCV', 'nestedCV_singleLoop', 'full_nestedCV', 'consensus_nestedCV']

In [ ]:
#----model----
model = RF # KNN, GB, RF, SVM
#----config----
config = get_config(os.path.join(ML_HP_DIR, f'{model.__name__}.json'))
print(f'config: \n{config}')
#---- data: MLData object ----
fpath = os.path.join(FEAT_LHD_OR_DIR, 'rmvD0', 'CHEMBL233_agon_G-GTP_EC50_CHEMBL3888720_lhd_b50_b50_featurized.pkl')

data = MLData(fpath)

descriptor = 'ECFP4'
aim = 'vs'
rmvS = 0
spl = 'rs_vs'
data(descriptor, rmvS, aim, spl)
data.get_int_splits()

#----run holdout CV----
metrics = consensus_nested_cv(config, model, data, save_config=True, save_model=True, verbose=True,
                            SPL='int', position=None)
print(f'Holdout CV metrics: {metrics}')

config: 
{'n_estimators': [100, 250, 500, 1000]}
Generating outer splits for int.rmvS0_rs_vs...
minClass_count: 25
minClass_count: 25
minClass_count: 25
minClass_count: 25
minClass_count: 26
max_minClass_count: 26
Generating inner splits for int.rmvS0_rs_vs...
min_class_count: 26
Inner train folds: [[0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 14, 15, 17, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51], [1, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 44, 46, 47, 48, 49, 50], [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 22, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 47, 48, 49, 51], [0, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 33, 35, 37, 38, 39, 41, 42, 43, 45, 46, 48, 49, 50, 51], [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19

## Benchmark

In [ ]:
# ===== model =====
algos = [RF, GB, SVM, KNN]
algos = [RF] # Yu: for testing purpose
# ===== data =====
# ----- directory -----
input_dirs = [FEAT_HHD_OR_DIR, FEAT_MHD_effect_OR_DIR, FEAT_MHD_OR_DIR, FEAT_LHD_OR_DIR]

rmvDs = [0, 1]
# ----- file -----
# identifiers in filename
# ----- x -----
descriptors = ['ECFP4','ECFP6','MACCS','RDKITFP','PHARM2D','ERG',
               'PHYSICOCHEM',
               'SHAPE3D','AUTOCORR3D','RDF','MORSE','WHIM','GETAWAY'
             ]
descriptors = ['ECFP4'] # Yu: for testing purpose
# ----- y -----
# aims = ['vs', 'lo']
# ----- split columns -----
SPLs = ['int', 'aln']
rmvSs = [0, 1]
aim_spl_combinations = {
    'lo': ['rs-lo', 'cs'], # when aim is 'lo', the test set would have similar structures as those in the training set. Thus, 'cs', i.e. cluster-stratified splitting is suitable here.
    'vs': ['rs-vs', 'ch']
}

# ----- pipelines -----




count = internal_benchmark(bmk_file='ml_internal_benchmark.csv')

pd_cd_pairs is 
[('hhd', 'mhd'), ('hhd', 'lhd'), ('mhd-effect', 'mhd'), ('mhd-effect', 'lhd'), ('mhd', 'lhd')]
ds_path: feat_hhd_or
descriptor: ECFP4, aim: lo, rmvS: 0, spl: rs-lo


KeyError: 'ECFP4'

In [ ]:
from datacat4ml.Scripts.data_prep.data_split.alnSplit_mldata import get_pd_cd_pairs, alignment_map, dir_name_dict, get_pfp_cfps_all

pd_cd_pairs = get_pd_cd_pairs(alignment_map)
print(f'pd_cd_pairs: \n{pd_cd_pairs}')
# pd_cd_pairs = [('hhd', 'mhd'), 
#                ('hhd', 'lhd'), 
#                ('mhd-effect', 'mhd'), 
#                ('mhd-effect', 'lhd'), 
#                ('mhd', 'lhd')]
pfp_cfps_all = get_pfp_cfps_all(rmvD=1)
print(f'pfps_cfps_all: \n{pfp_cfps_all}')
#pfp_cfps_all: dict. A static variable defined in alnSplit_mldata.py
#    during aligned splitting, the mapping from parent files to child files. 
#    i.e. {('hhd', 'mhd'):
#            {'CHEMBL233_None_None_Ki_None_hhd_b50_b50_split.csv': 
#                ['CHEMBL233_antag_G-GTP_Ki_None_mhd_b50_b50_split.csv', 
#                'CHEMBL233_bind_RBA_Ki_None_mhd_b50_b50_split.csv'], 
#                }
#          ...
#          }

for pd_cd_pair, pfp_cfps_map in pfp_cfps_all.items():
    print(f'pd_cd_pair: {pd_cd_pair}\n')
    print(f'pd_cat_level: {pd_cd_pair[0]}, cd_cat_level: {pd_cd_pair[1]}') # e.g. hhd,mhd

    pd_cat_level, cd_cat_level = pd_cd_pair
    print(f'pd_cat_level: {pd_cat_level}, cd_cat_level: {cd_cat_level}')
    
    for pf, cfs in pfp_cfps_map.items():
        print(f'parent file: {pf}, \nchild files: \n{cfs}')

        if len(cfs) == 0:
            print('No child files found, skip this pair.')
        else:
            for cf in cfs:
                print(f'cf to be processed: \n{cf}')
        print('---------------------------------')
    print('#################################')

pd_cd_pairs: 
[('hhd', 'mhd'), ('hhd', 'lhd'), ('mhd-effect', 'mhd'), ('mhd-effect', 'lhd'), ('mhd', 'lhd')]
pfps_cfps_all: 
{('hhd', 'mhd'): {'CHEMBL233_None_None_Ki_None_hhd': ['CHEMBL233_antag_G-GTP_Ki_None_mhd', 'CHEMBL233_bind_RBA_Ki_None_mhd'], 'CHEMBL236_None_None_IC50_None_hhd': ['CHEMBL236_bind_RBA_IC50_None_mhd', 'CHEMBL236_agon_G-cAMP_IC50_None_mhd', 'CHEMBL236_antag_B-arrest_IC50_None_mhd', 'CHEMBL236_antag_G-GTP_IC50_None_mhd'], 'CHEMBL233_None_None_IC50_None_hhd': ['CHEMBL233_antag_B-arrest_IC50_None_mhd', 'CHEMBL233_agon_G-cAMP_IC50_None_mhd', 'CHEMBL233_bind_RBA_IC50_None_mhd', 'CHEMBL233_antag_G-GTP_IC50_None_mhd'], 'CHEMBL237_None_None_Ki_None_hhd': ['CHEMBL237_antag_G-GTP_Ki_None_mhd', 'CHEMBL237_bind_RBA_Ki_None_mhd'], 'CHEMBL2014_None_None_EC50_None_hhd': ['CHEMBL2014_agon_G-cAMP_EC50_None_mhd', 'CHEMBL2014_agon_G-Ca_EC50_None_mhd', 'CHEMBL2014_agon_G-GTP_EC50_None_mhd'], 'CHEMBL236_None_None_EC50_None_hhd': ['CHEMBL236_agon_B-arrest_EC50_None_mhd', 'CHEMBL236_agon